In [1]:
import json

In [2]:
import graphlab as gl

In [3]:
import requests

In [4]:
from requests_file import FileAdapter

s = requests.Session()
s.mount('file://', FileAdapter())

resp = s.get('file:///users/ty/code/getmoore/data/us-west-1-spot.json')

In [5]:
pricing_history = resp.json()

In [6]:
pricing_history = pricing_history['SpotPriceHistory']

In [24]:
pricing_history[0]

{u'AvailabilityZone': u'us-west-1a',
 u'InstanceType': u'c4.large',
 u'ProductDescription': u'SUSE Linux',
 u'SpotPrice': u'0.120100',
 u'Timestamp': u'2016-09-21T00:50:37.000Z'}

In [7]:
sf = gl.SFrame(data=pricing_history)

This non-commercial license of GraphLab Create for academic use is assigned to ty.walker@gmail.com and will expire on November 24, 2016.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1474502342.log


In [30]:
sf.head()

X1
"{'Timestamp': '2016-09-21T00:50:37.000Z', ..."
"{'Timestamp': '2016-09-21T00:50:37.000Z', ..."
"{'Timestamp': '2016-09-21T00:50:36.000Z', ..."
"{'Timestamp': '2016-09-21T00:50:36.000Z', ..."
"{'Timestamp': '2016-09-21T00:50:36.000Z', ..."
"{'Timestamp': '2016-09-21T00:50:35.000Z', ..."
"{'Timestamp': '2016-09-21T00:50:35.000Z', ..."
"{'Timestamp': '2016-09-21T00:50:35.000Z', ..."
"{'Timestamp': '2016-09-21T00:50:32.000Z', ..."
"{'Timestamp': '2016-09-21T00:50:32.000Z', ..."


In [35]:
counts = {'Timestamp': 0, 'InstanceType': 0, 'ProductDescription': 0, 'SpotPrice': 0, 'Timestamp': 0}
for spot in pricing_history:
    keys = spot.keys()
    for key in keys:
        if key not in counts.keys():
            counts[key] = 0
        else:
            counts[key] += 1

In [36]:
counts

{u'AvailabilityZone': 2369078,
 'InstanceType': 2369079,
 'ProductDescription': 2369079,
 'SpotPrice': 2369079,
 'Timestamp': 2369079}

In [38]:
# All but one of the rows has the 'AvailabilityZone' key

In [8]:
timestamp, instance_type, product_description, spot_price, availability_zone = [], [], [], [], []

In [9]:
for spot in pricing_history:
    keys = spot.keys()
    if 'AvailabilityZone' not in keys:
        availability_zone.append(None)
    else:
        timestamp.append(spot['Timestamp'])
        instance_type.append(spot['InstanceType'])
        product_description.append(spot['ProductDescription'])
        spot_price.append(spot['SpotPrice'])
        availability_zone.append(spot['AvailabilityZone'])

In [43]:
len(timestamp)

2369079

In [44]:
len(instance_type)

2369079

In [45]:
len(product_description)

2369079

In [46]:
len(spot_price)

2369079

In [47]:
len(availability_zone)

2369079

In [10]:
# Load the SFrame. Note, changing SpotPrice to Price as this dataset only contains spot prices
sf = gl.SFrame({'Timestamp': timestamp, 'InstanceType': instance_type, 
                'ProductDescription': product_description, 'Price': spot_price, 
                'AvailabilityZone': availability_zone})

In [11]:
sf.head()

AvailabilityZone,InstanceType,Price,ProductDescription,Timestamp
us-west-1a,c4.large,0.120100,SUSE Linux,2016-09-21T00:50:37.000Z
us-west-1a,c4.large,0.020100,Linux/UNIX,2016-09-21T00:50:37.000Z
us-west-1b,m2.4xlarge,0.213300,SUSE Linux,2016-09-21T00:50:36.000Z
us-west-1b,m2.4xlarge,0.113300,Linux/UNIX,2016-09-21T00:50:36.000Z
us-west-1b,c4.large,0.100300,Windows,2016-09-21T00:50:36.000Z
us-west-1b,c3.2xlarge,0.265600,Windows,2016-09-21T00:50:35.000Z
us-west-1b,c3.2xlarge,0.254200,SUSE Linux,2016-09-21T00:50:35.000Z
us-west-1b,c3.2xlarge,0.154200,Linux/UNIX,2016-09-21T00:50:35.000Z
us-west-1b,r3.2xlarge,0.850000,SUSE Linux,2016-09-21T00:50:32.000Z
us-west-1b,r3.2xlarge,0.750000,Linux/UNIX,2016-09-21T00:50:32.000Z


In [50]:
gl.canvas.set_target('browser')

In [52]:
sf.show()

Canvas is accessible via web browser at the URL: http://localhost:52382/index.html
Opening Canvas in default web browser.


In [12]:
# See if c4.8xlarge was ever available for under $0.10 / hr
c48xlarge = sf[sf['InstanceType'] == 'c4.8xlarge']

In [13]:
c48xlarge.head()

AvailabilityZone,InstanceType,Price,ProductDescription,Timestamp
us-west-1b,c4.8xlarge,0.487000,SUSE Linux,2016-09-21T00:48:48.000Z
us-west-1b,c4.8xlarge,0.387000,Linux/UNIX,2016-09-21T00:48:48.000Z
us-west-1b,c4.8xlarge,1.587300,Windows,2016-09-21T00:48:48.000Z
us-west-1b,c4.8xlarge,1.587200,Windows,2016-09-21T00:48:20.000Z
us-west-1b,c4.8xlarge,0.485900,SUSE Linux,2016-09-21T00:48:19.000Z
us-west-1b,c4.8xlarge,0.385900,Linux/UNIX,2016-09-21T00:48:19.000Z
us-west-1a,c4.8xlarge,1.648100,Windows,2016-09-21T00:47:25.000Z
us-west-1b,c4.8xlarge,1.587300,Windows,2016-09-21T00:47:25.000Z
us-west-1a,c4.8xlarge,0.538400,SUSE Linux,2016-09-21T00:47:25.000Z
us-west-1b,c4.8xlarge,0.487000,SUSE Linux,2016-09-21T00:47:25.000Z


In [55]:
min(c48xlarge['Price'])

'0.256000'

Yikes, need to up that hashrate!

In [56]:
c48xlarge.show()

Canvas is accessible via web browser at the URL: http://localhost:52382/index.html
Opening Canvas in default web browser.


In [58]:
sf.save('data/us-west-1-spot-clean.json', format='json')
sf.save('data/us-west-1-spot-clean.csv', format='csv')

In [1]:
import seaborn

In [20]:
c48xlarge_price_time=c48xlarge['Timestamp', 'Price']

In [21]:
c48xlarge_price_time.head()

Timestamp,Price
2016-09-21T00:48:48.000Z,0.487000
2016-09-21T00:48:48.000Z,0.387000
2016-09-21T00:48:48.000Z,1.587300
2016-09-21T00:48:20.000Z,1.587200
2016-09-21T00:48:19.000Z,0.485900
2016-09-21T00:48:19.000Z,0.385900
2016-09-21T00:47:25.000Z,1.648100
2016-09-21T00:47:25.000Z,1.587300
2016-09-21T00:47:25.000Z,0.538400
2016-09-21T00:47:25.000Z,0.487000


In [23]:
len(c48xlarge_price_time['Timestamp'])

64531

In [24]:
len(c48xlarge_price_time['Price'])

64531

In [22]:
# This doesn't work because apparently AWS is adjusting their spot instance pricing multiple times per second
seaborn.tsplot(c48xlarge_price_time, time='Timestamp', value='Price')

ValueError: arrays must all be same length

In [28]:
len(c48xlarge_price_time)

64531

In [29]:
c48_pricing = c48xlarge_price_time.dropna()

In [30]:
len(c48_pricing)

64531

In [32]:
# Make sure we have the correct types or seaborn will break
c48_pricing['Price'].astype(float)

dtype: float
Rows: 64531
[0.487, 0.387, 1.5873, 1.5872, 0.4859, 0.3859, 1.6481, 1.5873, 0.5384, 0.487, 0.4384, 0.387, 0.4859, 0.3859, 1.5872, 1.5873, 0.487, 0.387, 0.4859, 0.3859, 0.5317, 0.4849, 0.4317, 0.3849, 1.6453, 0.4839, 0.3839, 0.4849, 0.3849, 0.4859, 0.3859, 1.5872, 0.533, 0.487, 0.433, 0.387, 1.6458, 1.5873, 0.4859, 0.3859, 1.5872, 1.6453, 0.5317, 0.4317, 1.5873, 0.487, 0.387, 0.4859, 0.3859, 0.4849, 0.3849, 1.6458, 0.533, 0.4839, 0.433, 0.3839, 0.4849, 0.3849, 1.6464, 0.5344, 0.4344, 0.4859, 0.3859, 0.533, 1.6458, 0.433, 1.5872, 0.4849, 0.3849, 1.6453, 0.5317, 0.4891, 0.4317, 0.3891, 1.5874, 0.4902, 0.3902, 0.4913, 0.3913, 1.5875, 1.6447, 0.5305, 0.4737, 0.4305, 0.3737, 0.4746, 0.3746, 1.6453, 0.5317, 0.4317, 0.4737, 0.3737, 0.5305, 0.4746, 0.4305, 0.3746, 1.6447, 0.5317, 0.4317, 1.6453, ... ]

In [36]:
c48_pricing['Price'] = c48_pricing['Price'].astype(float)

In [50]:
from datetime import datetime

In [41]:
test = c48_pricing['Timestamp'][0]

In [42]:
test

'2016-09-21T00:48:48.000Z'

In [45]:
t = test[:-5].replace('T', ' ')
t

'2016-09-21 00:48:48'

In [46]:
def clean_time(d):
    d = d[:-5].replace('T', ' ')
    return datetime.strptime(d, '%Y-%m-%d %H:%M:%S')

In [51]:
c48_pricing['Timestamp'] = c48_pricing['Timestamp'].apply(clean_time)

In [52]:
c48_pricing.head()

Timestamp,Price
2016-09-21 00:48:48,0.487
2016-09-21 00:48:48,0.387
2016-09-21 00:48:48,1.5873
2016-09-21 00:48:20,1.5872
2016-09-21 00:48:19,0.4859
2016-09-21 00:48:19,0.3859
2016-09-21 00:47:25,1.6481
2016-09-21 00:47:25,1.5873
2016-09-21 00:47:25,0.5384
2016-09-21 00:47:25,0.487


In [54]:
# You forgot to specify a specific location (us-west-1b) as an example :(
seaborn.heatmap(c48_pricing)

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''